## Background

We are going to build a model that does classifies customer reviews as positive or negative sentiment, using the [Women's E-Commerce Clothing Reviews Dataset](https://www.kaggle.com/datasets/nicapotato/womens-ecommerce-clothing-reviews).  We will walk you through how we would organize this task in Metaflow.  Concretely, we will demonstrate the following steps:

1. Read data from a parquet file
2. Show a branching workflow to record a baseline and train a model in parallel.  
3. Evaluate The Model on a holdout set 
4. Retrieve the correct version of our model artifacts to perform inference and write our predictions to a parquet file.

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.utils import set_random_seed
from tensorflow.keras import layers, optimizers, regularizers
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
set_random_seed(2022)

## Dataset 

Before we get started with metaflow, lets take a brief look at the dataset.  We have a table of reviews with a label of either `0` or `1` that means:

- 0: Negative sentiment
- 1: Positive sentiment

In [3]:
df = pd.read_parquet('train.parquet')
print(f'num of rows: {df.shape[0]}')

num of rows: 20377


In [4]:
df.head()

,labels,review
0,0,Odd fit: I wanted to love this sweater but the...
1,1,Very comfy dress: The quality and material of ...
2,0,Fits nicely but fabric a bit thin: I ordered t...
3,1,"Great fit: Love these jeans, fit and style... ..."
4,0,"Stretches out, washes poorly. wish i could ret..."


## Baseline Metrics

In this dataset, the most commmon label is postive sentiment.  Our baseline model should be able to beat the baseline

In [5]:
baseline_predictions = [1] * df.shape[0] # always predict the most prevalent class

print(f'Baseline Acccuracy: {accuracy_score(df.labels, baseline_predictions):.2%}')
print(f'Baseline AUC: {roc_auc_score(df.labels, baseline_predictions)}')

Baseline Acccuracy: 77.04%
Baseline AUC: 0.5


## Train Model

Preprocessing

In [6]:
cv = CountVectorizer(min_df=.005, max_df = .75, stop_words='english', strip_accents='ascii', )

In [7]:
res = cv.fit_transform(df['review'])
print(len(cv.vocabulary_))

766


 A Nueral-Bag-Of-Words Model

In [8]:
inputs = tf.keras.Input(shape=(len(cv.vocabulary_),), name='Input')
x = layers.Dropout(0.10)(inputs)
x = layers.Dense(15, activation="relu", kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4))(x)
predictions = layers.Dense(1, activation="sigmoid",)(x)
model = tf.keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
opt = optimizers.Adam(learning_rate=0.002)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

2022-07-19 15:27:53.892958: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
model.fit(x=res.toarray(), 
          y=df['labels'],
          batch_size=32, epochs=10, validation_split=.2)

Epoch 1/10
510/510 [==============================] - 1s 1ms/step - loss: 0.3523 - accuracy: 0.8507 - val_loss: 0.2988 - val_accuracy: 0.8754
Epoch 2/10
510/510 [==============================] - 1s 996us/step - loss: 0.2945 - accuracy: 0.8818 - val_loss: 0.2956 - val_accuracy: 0.8771
Epoch 3/10
510/510 [==============================] - 1s 985us/step - loss: 0.2860 - accuracy: 0.8840 - val_loss: 0.2989 - val_accuracy: 0.8768
Epoch 4/10
510/510 [==============================] - 1s 991us/step - loss: 0.2761 - accuracy: 0.8891 - val_loss: 0.2951 - val_accuracy: 0.8803
Epoch 5/10
510/510 [==============================] - 1s 985us/step - loss: 0.2676 - accuracy: 0.8956 - val_loss: 0.2991 - val_accuracy: 0.8759
Epoch 6/10
510/510 [==============================] - 1s 1ms/step - loss: 0.2623 - accuracy: 0.9005 - val_loss: 0.2996 - val_accuracy: 0.8793
Epoch 7/10
510/510 [==============================] - 1s 1ms/step - loss: 0.2549 - accuracy: 0.9056 - val_loss: 0.3044 - val_accuracy: 0.875

## Make Predictions

These are toy examples:

In [22]:
review = ["poor fit its baggy in places where it isn't supposed to be.", # should be negative
          "love it, very high quality and great value"]                  #should be positive
toy_preds = model.predict(cv.transform(review).toarray())
toy_preds

array([[0.02719706],
       [0.9402847 ]], dtype=float32)

In [24]:
toy_preds[0][0] < .5 and 

True

## Evaluate Model On Holdout Set

In [11]:
hdf = pd.read_parquet('predict.parquet')
predictions = model.predict(cv.transform(hdf['review']).toarray())
labels = hdf['labels']

In [12]:
accuracy_score(labels, predictions>.5)

0.8776501766784452

In [25]:
roc_auc_score(labels, predictions)

NameError: name 'labels' is not defined

In [26]:
pd.concat([df, df])

,labels,review
0,0,Odd fit: I wanted to love this sweater but the...
1,1,Very comfy dress: The quality and material of ...
2,0,Fits nicely but fabric a bit thin: I ordered t...
3,1,"Great fit: Love these jeans, fit and style... ..."
4,0,"Stretches out, washes poorly. wish i could ret..."
...,...,...
20372,1,Love: This sweater just arrived in my mailbox ...
20373,1,Classic and sophisticated: I recently purchase...
20374,1,nan: This tunic is a wonderfully updated look....
20375,1,Great staple: I've ordered a pair of these fro...


In [27]:
?model.re

Signature: model.reset_states()
Docstring: <no docstring>
File:      ~/opt/anaconda3/envs/full-stack-metaflow/lib/python3.8/site-packages/keras/engine/training.py
Type:      method


In [28]:
from tensorflow.keras import K

ImportError: cannot import name 'K' from 'tensorflow.keras' (/Users/hamel/opt/anaconda3/envs/full-stack-metaflow/lib/python3.8/site-packages/keras/api/_v2/keras/__init__.py)